In [1]:
import numpy as np
import math, random
from  scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from Spatial_distributions import *
from Independent_functions import *
from Staples_related import *

In [192]:
class Group:
    def __init__(self, xyz, names):
        self.xyz = xyz
        self.names = names
    def save_group(self, file):
        for i in range(len(self.xyz)):
            file.write("{:<6}{:>12.4f}{:>12.4f}{:>12.4f}\n".format(self.names[i], self.xyz[i,0], self.xyz[i,1], self.xyz[i,2]))

            
class Molecule:
    def __init__(self, names, atids):
        self.names = names
        self.atids = np.array(atids)
        self.nat = len(names)
        
class Frame:
    def __init__(self, xyz, names, time):
        self.xyz = np.array(xyz)
        self.names = names
        self.time = time
        self.all = Group(self.xyz, names)
        
    def displace_box(self, bx, by, bz, vec):
        #bx, by, bz, and vec are given in angs
        self.xyz = self.xyz - vec
        self.check_boundary(bx, 0)
        self.check_boundary(by, 1)
        self.check_boundary(bz, 2)
        self.update_all_group()
        
    def check_boundary(self, B, dim):
        #B must be given in angs
        ndx = np.where(self.xyz[:,dim] < 0)[0]
        if len(ndx) > 0:
            self.xyz[ndx,dim] = self.xyz[ndx,dim] + (B) #plus sign because the coordinates are already negative
        ndx = np.where(self.xyz[:,dim] > (B))[0]
        if len(ndx) > 0:
            self.xyz[ndx,dim] = self.xyz[ndx,dim] - (B)
        
    def update_all_group(self):
        self.all = Group(self.xyz, self.names)

class Trajectory:
    def __init__(self, directory, skip = 1, dt = 0):
        #Imports molecules from fort.5 file
        self.molecules = []
        f5 = np.genfromtxt(directory + "fort.5", delimiter = "\n", dtype = 'str')
        inis = []
        fins = []
        molids = []
        for l, line in enumerate(f5):
            if "molecola nr." in line:
                inis.append(l+2)
                fins.append(l)  
                molids.append(line.split()[2])
        fins.pop(0)
        fins.append(len(f5))
        blocks = []
        for i in range(len(inis)):
            ini = inis[i]
            fin = fins[i]
            blocks.append(f5[ini:fin])
        for b in blocks:
            names = []
            atids = []
            for line in b:
                name = line.split()[1]
                atid = int(line.split()[0])
                names.append(name)
                atids.append(atid)
            self.molecules.append(Molecule(names, atids))    
                
        #Imports trajectory from fort.8 file
        f8 = np.genfromtxt(directory + "fort.8", delimiter = "\n", dtype = 'str')
        box = np.array((f8[1].split())).astype('float')
        self.frames = []
        self.names = []
        self.N_beads = int(f8[0])
        self.BX, self.BY, self.BZ = box[1], box[2], box[3]
        self.N_frames = int(len(f8)/(self.N_beads+2))
        
        for i in range(self.N_frames):
            ini = i*(self.N_beads + 2) + 2
            fin = ini + self.N_beads
            info = f8[ini-1].split()
            time = float(info[0])
            
            if dt == 0:
                if i%skip == 0:
                    xyz = []
                    names = []
                    for j in range(ini, fin):
                        line = f8[j].split()
                        xyz.append(np.array(line[1:]).astype('float'))
                        names.append(line[0])
                    self.frames.append(Frame(xyz, names, time))
            else:
                if time%dt == 0:
                    xyz = []
                    names = []
                    for j in range(ini, fin):
                        line = f8[j].split()
                        xyz.append(np.array(line[1:]).astype('float'))
                        names.append(line[0])
                        self.names(line[0])
                    self.frames.append(Frame(xyz, names, time))  
    
    def displace_box(self, vec):
        vec = np.array(vec)
        for f in self.frames:
            f.displace_box(self.BX, self.BY, self.BZ, vec)
    
    def center_mol(self, molid):
        ndx = self.molecules[molid-1].atids - 1
        for f in self.frames:
            vec = np.mean(f.xyz[ndx], axis = 0) - np.array([self.BX, self.BY, self.BZ])/2
            f.displace_box(self.BX, self.BY, self.BZ, vec)
        
    def center_name(self, name):
        ndx = np.where(self.names == name)[0]
        for f in self.frames:
            vec = np.mean(f.xyz[ndx], axis = 0) - np.array([self.BX, self.BY, self.BZ])/2
            f.displace_box(self.BX, self.BY, self.BZ, vec)
    
    def align_withfile(file):
        
    def save_trj(self, fname):
        f = open(fname, "w")
        for i in range(len(self.frames)):
            f.write("{}\n".format(self.N_beads))
            f.write("{:>12.4f}{:>12.4f}{:>12.4f}{:>12.4f}\n".format(self.frames[i].time, self.BX, self.BY, self.BZ))
            self.frames[i].all.save_group(f)
        f.close()

In [193]:
directories = ["../../PHD/OCCAM/Mix1-7/"\
               ]
for d in directories:
    T = Trajectory(d, skip = 100)
    #T.displace_box([35, 35, 35])
    T.align_mol(1)
    T.save_trj(t + "f8_FIX.xyz")
    #T.align_mol(1)
    #T.save_trj(t + "f8_FIX_amol.xyz")

[9.89130435 9.89130435 9.89130435]
[54.7826087  11.41304348 21.30434783]
[14.45652174 18.26086957 50.2173913 ]
[15.2173913  15.97826087 22.82608696]
[25.10869565 60.10869565 15.97826087]
[37.2826087  19.7826087  12.93478261]
[54.7826087  26.63043478 31.19565217]
[25.86956522 28.15217391 46.41304348]
[11.41304348 27.39130435 38.80434783]
[32.7173913  30.43478261 35.        ]
[11.41304348 26.63043478 53.26086957]
[32.7173913  33.47826087 41.84782609]
[32.7173913  35.         45.65217391]
[26.63043478 34.23913043 39.56521739]
[32.7173913  28.15217391 36.52173913]
[34.23913043 26.63043478 48.69565217]
[35.         29.67391304 42.60869565]
[35.         24.34782609 41.84782609]
[35.         19.02173913 38.80434783]
[35.         26.63043478 44.13043478]
[35.         33.47826087 43.36956522]
[35.         16.73913043 36.52173913]
[35.         24.34782609 43.36956522]
[35.         20.54347826 38.80434783]
[35.         32.7173913  39.56521739]
[35.         30.43478261 35.76086957]
